In [105]:
import pandas as pd
import time
import numpy as np
import lyricsgenius

In [106]:
kpop_world_sales = pd.read_csv('Kpop on BB - World Digital Sales.csv')

kpop_world_sales.insert(3, 'original_lyrics', ['' for i in range(len(kpop_world_sales))])
kpop_world_sales.insert(4, 'translated_lyrics', ['' for i in range(len(kpop_world_sales))])

kpop_world_sales.rename(columns={'Chart date[169]': 'chart_date', 
                            'Artist': 'artist',
                            'Song/Album Title': 'song_title',
                            'Peak position': 'peak_position',
                            'Consecutive\nentry weeks': 'entry_weeks'}, inplace=True)
                            
##standardize and process chart data

#remove wiki citations, standardize whitespace, remove quotation marks
#regex patterns and replacement strings
regex_pats = [[r"(\[\d+\])", ""], [r"(\s)", " "], [r"(\")", ""]]
for pat in regex_pats:
    kpop_world_sales["artist"] = kpop_world_sales["artist"].str.replace(pat[0], pat[1])
    kpop_world_sales["song_title"] = kpop_world_sales["song_title"].str.replace(pat[0], pat[1])
    kpop_world_sales["peak_position"] = kpop_world_sales["peak_position"].str.replace(pat[0], pat[1])

#pull out just main artist, remove features (for genius search)
kpop_world_sales['main_artist'] = kpop_world_sales["artist"].str.replace(r"(feat.*)", '')
kpop_world_sales['main_artist'] = kpop_world_sales["main_artist"].str.replace(r"(and .*)", '')
kpop_world_sales['main_artist'] = kpop_world_sales["main_artist"].str.strip()

#standardize peak positions (remove parentheticals) + remove trailing whitespace + remove all non-digits
kpop_world_sales["peak_position"] = kpop_world_sales["peak_position"].str.replace(r"(\(.*\))", "")
kpop_world_sales["peak_position"] = kpop_world_sales["peak_position"].str.strip()
kpop_world_sales["peak_position"] = kpop_world_sales["peak_position"].str.replace(r"[a-z]", "")

In [107]:
##store best position if multiple are listed
for i, r in kpop_world_sales.iterrows():
    if len(r['peak_position'].split(' ')) > 1:
            ps = []
            for p in r['peak_position'].split(' '):
                try:
                    ps.append(int(p))
                except: pass
            print(ps, min(ps))
            kpop_world_sales.at[i, 'peak_position'] = min(ps)
    kpop_world_sales.at[i, 'peak_position'] = int(r['peak_position'])

[4, 1] 1
[4, 22] 4
[2, 1] 1
[1] 1
[1] 1


In [108]:
##narrow down songs that are included for now
top_songs = kpop_world_sales.loc[kpop_world_sales['peak_position'] <= 3]

In [126]:
##search for original lyrics on genius
genius = lyricsgenius.Genius('KQ5oZoRnw1ea0YwPjOeDFacmn37UlUDjp8DGKlj7pfgDsIXJYfOz00skKmR8hqhR')

In [127]:
for artist_name in top_songs['main_artist'].unique():
    print(artist_name)
    #locate all songs the artist has in the top songs
    artist_songs = top_songs.loc[top_songs['main_artist'] == artist_name]
    #count number of lyrics for this artist that haven't been filled in
    missing_lyrics = len(artist_songs.loc[artist_songs['original_lyrics'] == ''])
    #rerun genius if lyrics are missing
    if missing_lyrics > 1:
        try: #search for artist on genius, skip if not found
            #seearch for top songs for artist on genius, sorted by popularity -- should mostly match song chart positions
            artist_genius = genius.search_artist(artist_name, max_songs=len(artist_songs) + 5, sort="popularity")
            ##search for each song on genius and store lyrics
            for i,r in artist_songs.iterrows():
                print(r['song_title'])
                song_genius = artist_genius.song(r['song_title'])
                #store song lyrics if possible, otherwise skip
                try:
                    lyrics_raw = song_genius.lyrics
                    print(lyrics_raw[:30])
                    top_songs.at[i, 'original_lyrics'] = lyrics_raw
                except:
                    pass
        except:
            pass

Shinee
2NE1
2PM
Jaejoong
Big Bang
Jay Park
2:00 PM
miss A
Super Junior
Wonder Girls
IU
Block B
f(x)
Psy
Ultimate Pop Hits!
Girls' Generation
B.A.P
BTS
Searching for songs by BTS...

Changing artist name to 'BTS (방탄소년단)'
Song 1: "Dynamite"
Song 2: "FAKE LOVE"
Song 3: "MIC Drop (Steve Aoki Remix)"
Song 4: "전하지 못한 진심 (The Truth Untold)"
Song 5: "봄날 (Spring Day)"
Song 6: "DNA"
Song 7: "MIC Drop (Steve Aoki Remix) [Desiigner Remix]"
Song 8: "Intro: Serendipity (세렌디피티)"
Song 9: "Euphoria"
Song 10: "Airplane pt.2"
Song 11: "Magic Shop"
Song 12: "피 땀 눈물 (Blood Sweat & Tears)"
Song 13: "Crystal Snow"
Song 14: "MIC Drop"
Song 15: "Anpanman"
Song 16: "Intro: Singularity"
Song 17: "작은 것들을 위한 시 (Boy With Luv)"
Song 18: "BEGIN"
Song 19: "ON"
Song 20: "고민보다 Go (Go Go)"
Song 21: "땡 (Ddaeng)"
Song 22: "Best of Me"
Song 23: "낙원 (Paradise)"
Song 24: "Outro: Tear"
Song 25: "Not Today"
Song 26: "Love Maze"
Song 27: "LIE"
Song 28: "Don’t Leave Me"
Song 29: "Let Go"
Song 30: "134340"
Song 31: "Life Goes On"


In [145]:
for artist_name in top_songs['main_artist'].unique():
    print(artist_name)
    #locate all songs the artist has in the top songs
    artist_songs = top_songs.loc[top_songs['main_artist'] == artist_name]
    missing_lyrics = len(artist_songs.loc[artist_songs['original_lyrics'] == ''])
    print(missing_lyrics)

Shinee
0
2NE1
1
2PM
0
Jaejoong
0
Big Bang
0
Jay Park
0
2:00 PM
1
miss A
0
Super Junior
0
Wonder Girls
0
IU
0
Block B
0
f(x)
1
Psy
0
Ultimate Pop Hits!
1
Girls' Generation
0
B.A.P
0
BTS
9
Exo
0
Taeyang
0
T.O.P
0
VIXX
0
Sistar
0
Hi Suhyun
0
G-Dragon
1
4Minute
0
Red Velvet
0
Got7
0
iKON
0
Hyuna
0
Seventeen
0
Bae Suzy & Baekhyun
1
Taeyeon
0
Yoo Young-jin
0
Taemin
0
Mamamoo
0
K.Will
1
NCT U
0
Twice
10
NCT 127
0
Blackpink
0
NCT Dream
1
Bobby
0
MOBB
1
Yoo Jae-suk X Exo
0
Infinite
0
Zico
0
Park Chanyeol & Punch
1
Day6
0
BgA (*K-pop parody group)
1
Winner
0
K.A.R.D
2
Sunmi
0
BTS BTS
2
J-Hope
0
(G)I-dle
0
Oh!GG
0
Chanyeol & Sehun
0
Seulgi, SinB, Chungha, Soyeon
0
K/DA
1
Jennie
1
IKON
1
Lee So-ra
1
Itzy
0
Hwasa
1
TXT
0
Park Bom
1
Stray Kids
0
Everglow
0
Kard
1
SuperM
1
CL
1
Loona
0
Younha
0
Monsta X
1
Loona's JinSoul
0
Suga (Agust D)
0
K/DA (G)I-dle & Wolftyla
1
Verivery
0
K/DA (G)I-dle, Madison Beer, Jaira Burns, & Lexie Liu as Seraphine
1


In [134]:
#rerun genius search for blackpink with modified search term
genius = lyricsgenius.Genius('KQ5oZoRnw1ea0YwPjOeDFacmn37UlUDjp8DGKlj7pfgDsIXJYfOz00skKmR8hqhR')

artist_name = 'Blackpink'
#locate all songs the artist has in the top songs
artist_songs = top_songs.loc[top_songs['main_artist'] == artist_name]

#seearch for top songs for artist on genius, sorted by popularity -- should mostly match song chart positions
artist_genius = genius.search_artist('BLACKPINK', max_songs=len(artist_songs) + 5, sort="popularity")

##search for each song on genius and store lyrics
for i,r in artist_songs.iterrows():
    print(r['song_title'])
    song_genius = artist_genius.song(r['song_title'])
    #store song lyrics if possible, otherwise skip
    try:
        lyrics_raw = song_genius.lyrics
        print(lyrics_raw[:30])
        top_songs.at[i, 'original_lyrics'] = lyrics_raw
    except:
        pass

Searching for songs by BLACKPINK...

Song 1: "마지막처럼 (AS IF IT’S YOUR LAST)"
Song 2: "뚜두뚜두 (DDU-DU DDU-DU)"
Song 3: "Lovesick Girls"
Song 4: "Kill This Love"
Song 5: "How You Like That"
Song 6: "Pretty Savage"
Song 7: "Love To Hate Me"
Song 8: "Bet You Wanna"
Song 9: "Crazy Over You"
Song 10: "WHISTLE -Japanese Ver.-"
Song 11: "DDU-DU DDU-DU -JP Ver.-"
Song 12: "FOREVER YOUNG"
Song 13: "BOOMBAYAH -Japanese Ver.-"
Song 14: "Sure Thing (Miguel Cover)"

Reached user-specified song limit (14).
Done. Found 14 songs.
Boombayah
Searching for "Boombayah" by BLACKPINK...
Done.
[블랙핑크 "붐바야" 가사]

[Spoken: Jenn
Whistle
Searching for "Whistle" by BLACKPINK...
Done.
[블랙핑크 "휘파람" 가사]

[Spoken: Jiso
Playing With Fire
Searching for "Playing With Fire" by BLACKPINK...
Done.
[블랙핑크 "불장난" 가사]

[Verse 1: Jen
As If It's Your Last
Searching for "As If It's Your Last" by BLACKPINK...
Done.
[블랙핑크 "마지막처럼" 가사]

[Intro: Jen
Ddu-Du Ddu-Du
Searching for "Ddu-Du Ddu-Du" by BLACKPINK...
Done.
[블랙핑크 "뚜두뚜두" 가사]

[Intro: Al

In [136]:
#rerun genius search for AgustD with modified search term
genius = lyricsgenius.Genius('KQ5oZoRnw1ea0YwPjOeDFacmn37UlUDjp8DGKlj7pfgDsIXJYfOz00skKmR8hqhR')

artist_name = 'Suga (Agust D)'
#locate all songs the artist has in the top songs
artist_songs = top_songs.loc[top_songs['main_artist'] == artist_name]

#seearch for top songs for artist on genius, sorted by popularity -- should mostly match song chart positions
artist_genius = genius.search_artist('AGUST D', max_songs=len(artist_songs) + 5, sort="popularity")

##search for each song on genius and store lyrics
for i,r in artist_songs.iterrows():
    print(r['song_title'])
    song_genius = artist_genius.song(r['song_title'])
    #store song lyrics if possible, otherwise skip
    try:
        lyrics_raw = song_genius.lyrics
        print(lyrics_raw[:30])
        top_songs.at[i, 'original_lyrics'] = lyrics_raw
    except:
        pass

Searching for songs by AGUST D...

Changing artist name to 'Agust D'
Song 1: "Agust D"
Song 2: "마지막 (The Last)"
Song 3: "대취타 (Daechwita)"
Song 4: "So Far Away"
Song 5: "Tony Montana"
Song 6: "Give It To Me"
Song 7: "어떻게 생각해? (What do you think?)"
Song 8: "저 달 (Moonlight)"

Reached user-specified song limit (8).
Done. Found 8 songs.
Daechwita
Searching for "Daechwita" by Agust D...
Done.
[대취타 (Daechwita) 가사]

[Sample:
Strange
Searching for "Strange" by Agust D...
Done.
[이상하지 않은가 (Strange) 가사]

[Intr
Burn It
Searching for "Burn It" by Agust D...
Done.
[Burn It 가사]

[Chorus: MAX]
I 


In [151]:
translations = genius.search_artist('Genius English Translations', max_songs=1)

Searching for songs by Genius English Translations...

Song 1: "Luis Fonsi & Daddy Yankee - Despacito (Remix) ft. Justin Bieber (English Translation)"

Reached user-specified song limit (1).
Done. Found 1 songs.


In [156]:
for i, r in top_songs.iterrows():
    this_song = r['song_title']
    this_artist = r['main_artist']
    print(this_song + ' ' + this_artist)
    trans_song = translations.song(this_song + ' ' + this_artist)
    print(trans_song)
    if trans_song: ###




        #need to add test to make sure its getting the right search result
        # and not random proust chapters








        
        trans_lyrics = trans_song.lyrics
        print(trans_lyrics[:20])
        top_songs.at[i, 'translated_lyrics'] = trans_lyrics


Lucifer Shinee
Searching for "Lucifer Shinee" by Genius English Translations...
Done.
"SHINee - Lucifer (English Translation)" by Genius English Translations:
    Even if I try to avoid you
    I can't find a place to hide
    I can't even deny you
    I'm trapped by you
    If ...
Even if I try to avo
Can't Nobody 2NE1
Searching for "Can't Nobody 2NE1" by Genius English Translations...
No results found for: 'Can't Nobody 2NE1 Genius English Translations'
None
Hello Shinee
Searching for "Hello Shinee" by Genius English Translations...
Done.
"English Bards and Scotch Reviewers" by Lord Byron:
    English Bards And Scotch Reviewers [1]
    
    Still [2] must I hear?—shall hoarse [3] FITZGERALD bawl
    His ...
English Bards And Sc
I'll Be Back 2PM
Searching for "I'll Be Back 2PM" by Genius English Translations...
No results found for: 'I'll Be Back 2PM Genius English Translations'
None
To You It's Goodbye, To Me It's Waiting Jaejoong
Searching for "To You It's Goodbye, To Me It's Waitin

KeyboardInterrupt: 

In [129]:
top_songs.to_csv('TopSongSales_Kpop.csv')

NameError: name 'top_songs' is not defined